## Matriz de substituição de nucleotídeos: modelo K2P de Kimura, 1980.

### Calcula o scoring de substituição para match e mismatch entre as bases.

Similaridade = recompensa de +4

Transição = penalidade de -1

Transversão = penalidade de -2


```
   A  C  G  T
A  4 -2 -1 -2
C -2  4 -2 -1
G -1 -2  4 -2  
T -2 -1 -2  4  

```

Inserção ou deleção de base (gap) = penalidade de -3

In [142]:
# padronizar a sequência inserida pelo usuário (formato FASTA ou sequência livre de nucleotídeos).
# insere um * no início da sequência para construçã da matriz para o alinhamento.

def formata_sequencia(sequencia): 
    sequencia = sequencia.upper()
        
    if sequencia[0] == ">":
        sequencia = sequencia.splitlines()
        sequencia = sequencia[1:]
        sequencia = "".join(sequencia).strip()
        sequencia = "*" + sequencia
        
    else:
        sequencia = sequencia.splitlines()
        sequencia = "".join(sequencia).strip()
        sequencia = "*" + sequencia 
        
    return sequencia


In [143]:
# verifica se a sequência é de DNA

def eh_dna(seq):
    if set(seq).issubset({"A", "C", "G", "T", "*"}):
        return True
    else:
        return False

In [144]:
# criar a matriz de substituição

def cria_matriz_subs():

    matriz = {"col": ["A", "C", "G", "T"],
                "A": [4, -2, -1, -2],
                "C": [-2, 4, -2, -1],
                "G": [-1, -2, 4, -2],
                "T": [-2, -1, -2, 4]}

    return matriz

In [145]:
# retorna o score do match ou mismatch entre 2 bases pareadas

def calcula_score(base1, base2, matriz_subs):

    j = matriz_subs["col"].index(base1)
    for base in "ACGT":
        if base2 == base:
            score = matriz_subs[base2][j]

            return score

In [146]:
# retorna o maior valor do L 

def valor_maximo(base1, base2, lado, cima, diagonal):
    # match
    if (base1 == base2) and (diagonal > lado) and (diagonal > cima):
        return diagonal
    # mismatch
    elif (base1 != base2) and (diagonal >= lado) and (diagonal >= cima):
        return diagonal
    # gap
    elif (lado > cima) and (lado > diagonal):
        return lado
    # gap
    else:
        return cima

In [147]:
imprime_matriz(seq1, seq2, matriz_pontuacao, matriz_caminho)

	*	A	T	C	G	A	
*	0-	-3-	-6-	-9-	-12-	-15-	
A	-3|	4\	1-	-2-	-5-	-18|	
A	-6|	1|	2\	-1|	-3\	-1\	
C	-9|	-2|	0\	6\	3-	0-	
G	-12|	-5|	-3|	3|	10\	7-	
G	-15|	-8|	-6|	0|	7|	9\	
G	-18|	-11|	-9|	-3|	4|	6|	
A	-21|	-14|	-12|	-6|	1|	8\	



In [148]:
# retorna de onde veio o maior valor do L

def cria_caminho(base1, base2, lado, cima, diagonal):
    # match
    if (base1 == base2) and (diagonal > lado) and (diagonal > cima):
        return "\\"
    # mismatch
    elif (base1 != base2) and (diagonal > lado) and (diagonal > cima):
        return "\\"
    # gap
    elif (lado > cima) and (lado > diagonal):
        return "-"
    # gap
    else:
        return "|"

In [149]:
# lcs = longest common subsequence
# montar 2 matrizes:
# pontuacao: receberá a pontuação (score) de cada par de base analisado
# caminho: recebe o caminho do qual a pontuação veio

def lcs(seq1, seq2, matriz_subs):

    pontuacao = []
    caminho = []
    g = -3

    # preencher a matriz
    for i in range(0, len(seq1)):
        pontuacao.append([0] * len(seq2))
        caminho.append([""] * len(seq2))

    # preencher a primeira coluna 
    for i in range(0, len(seq1)):
        pontuacao[i][0] = -3 * i
        caminho[i][0] = "|"
    # preenche a primeira linha
    for j in range(0, len(seq2)):
        pontuacao[0][j] = -3 * j
        caminho[0][j] = "-"

    # linha
    for i in range(1, len(seq1)):
        # coluna
        for j in range(1, len(seq2)):
        # devolver a valor maximo do L

            base1 = seq1[i]
            base2 = seq2[j] 

            # calcula o score (match e mismatch)
            s = calcula_score(base1, base2, matriz_subs)
            
            lado = pontuacao[i][j-1]
            cima = pontuacao[i-1][j]
            diagonal = pontuacao[i-1][j-1]
                                    
            pontuacao[i][j] = valor_maximo(base1, base2, lado + g, cima + g, diagonal + s)
            caminho[i][j] = cria_caminho(base1, base2, lado + g, cima + g, diagonal + s)

    return pontuacao, caminho

In [150]:
# imprime a matriz com a pontuação e o caminho

def imprime_matriz(seq1, seq2, matriz_pontuacao, matriz_caminho):

    print("\t", end="")

    for j in range(0, len(seq2)):
        print(seq2[j], end="\t")
    print()
    for i in range(0, len(seq1)):
        print(seq1[i], end="\t")

        for j in range(0, len(seq2)):
            print(matriz_pontuacao[i][j], matriz_caminho[i][j], end="\t", sep="")
        print()
    print()

In [151]:
# imprime o alinhamento ótimo entre as sequências

def gera_alinhamento(seq1, seq2, ponteiros, matriz_subs):
    
    ali_seq1 = ""
    ali_seq2 = ""
    
    match = 0
    mismatch = 0
    gap = 0
    score_final = 0
    g = -3

    # linhas
    i = len(seq1)-1
    # colunas
    j = len(seq2)-1

    while (i != 0) or (j != 0):

        s = calcula_score(seq1[i], seq2[j], matriz_subs)

        if matriz_caminho[i][j] == "\\" and seq1[i] == seq2[j]:
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            match += 1
            score_final += s
            i -= 1
            j -= 1

        elif matriz_caminho[i][j] == "\\" and seq1[i] != seq2[j]:
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            mismatch += 1
            score_final += s
            i -= 1
            j -= 1    
    
        elif matriz_caminho[i][j] == "-":
            ali_seq1 = "-" + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            gap += 1
            score_final += g
            j -= 1
    
        elif matriz_caminho[i][j] == "|":
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = "-" + ali_seq2
            gap += 1
            score_final += g
            i -= 1

    print(ali_seq1)  
    print(ali_seq2) 
    print()
    print(f"Matches = {match}")
    print(f"Mismatches = {mismatch}")
    print(f"Gaps = {gap}")
    print(f"Score final = {score_final}") 

In [155]:
seqA = "AAATTTACGTGGTT"
seqB = "ACGTGT"

seq1 = formata_sequencia(seqA)
seq2 = formata_sequencia(seqB)

In [156]:
matriz_subs = cria_matriz_subs()
matriz_pontuacao, matriz_caminho = lcs(seq1, seq2, matriz_subs)
gera_alinhamento(seq1, seq2, matriz_caminho, matriz_subs)

AAATTTACGTGGTT
A------CGTG-T-

Matches = 6
Mismatches = 0
Gaps = 8
Score final = 0


In [157]:
imprime_matriz(seq1, seq2, matriz_pontuacao, matriz_caminho)

	*	A	C	G	T	G	T	
*	0-	-3-	-6-	-9-	-12-	-15-	-18-	
A	-3|	4\	1-	-2-	-5-	-8-	-11-	
A	-6|	1|	2\	0\	-3-	-6|	-9-	
A	-9|	-2|	-1|	1\	-2|	-4\	-7-	
T	-12|	-5|	-3\	-2|	5\	2-	0\	
T	-15|	-8|	-6|	-5|	2|	3\	6\	
T	-18|	-11|	-9|	-8|	-1|	0|	7\	
A	-21|	-14|	-12|	-10\	-4|	-2\	4|	
C	-24|	-17|	-10\	-13|	-7|	-5|	1|	
G	-27|	-20|	-13|	-6\	-9-	-3\	-2|	
T	-30|	-23|	-16|	-9|	-2\	-5-	1\	
G	-33|	-26|	-19|	-12|	-5|	2\	-1-	
G	-36|	-29|	-22|	-15|	-8|	-1|	0\	
T	-39|	-32|	-25|	-18|	-11|	-4|	3\	
T	-42|	-35|	-28|	-21|	-14|	-7|	0|	

